### CREATE FACT TABLE

**Read Silver data**

In [0]:
df_silver = spark.sql(
    """
select * from parquet.`abfss://silver@claimcraftadls.dfs.core.windows.net/Claimcraft_Cleaned_data`"""
)

display(df_silver)

#### Read from all Dimension table

In [0]:
df_product = spark.sql("select * from `pharma_claim_craft`.claim_craft_gold.dim_product")
df_pharmacy = spark.sql("select * from `pharma_claim_craft`.claim_craft_gold.dim_pharmacy")
df_channel= spark.sql("select * from `pharma_claim_craft`.claim_craft_gold.dim_channel")
df_patient_zip = spark.sql("select * from `pharma_claim_craft`.claim_craft_gold.dim_patient_zip")
df_notes = spark.sql("select * from `pharma_claim_craft`.claim_craft_gold.dim_notes")
df_plan = spark.sql("select * from `pharma_claim_craft`.claim_craft_gold.dim_plan")
df_rep = spark.sql("select * from `pharma_claim_craft`.claim_craft_gold.dim_rep")
df_submission= spark.sql("select * from `pharma_claim_craft`.claim_craft_gold.dim_submission")

In [0]:
df_fact = (
    df_silver.join(
        df_product, df_silver["Product_NDC"] == df_product["Product_NDC"], "left"
    ).join(
        df_pharmacy, df_silver["Pharmacy_Name"] == df_pharmacy["Pharmacy_Name"], "left"
    ).join(
        df_channel, df_silver["Channel"] == df_channel["Channel"], "left"
    ).join(
        df_notes, df_silver["Notes"] == df_notes["Notes"], "left"
    ).join(
        df_plan, 
        (df_silver["Plan_Type"] == df_plan["Plan_Type"]) & 
        (df_silver["Coverage_Status"] == df_plan["Coverage_Status"]) &
        (df_silver["Prior_Auth"] == df_plan["Prior_Auth"]),
        "left"
    ).join(
        df_submission, df_silver["Submission_Source"] == df_submission["Submission_Source"], "left"
    ).join(
        df_patient_zip, df_silver["Patient_Zip"] == df_patient_zip["Patient_Zip"], "left"
    ).join(
        df_rep,
        (df_silver["Rep_ID"] == df_rep["Rep_ID"]) & 
        (df_silver["Territory"] == df_rep["Territory"]),
        "left"
    ).select(
        df_silver["Claim_ID"],
        df_silver["Claim_Date"],
        df_silver["Copay_Amount"],
        df_silver["Billed_Amount"],
        df_silver["Paid_Amount"],
        df_silver["Duplicate_Claim"],
        df_silver["Total_Claim"],
        df_silver["Copay_Percentage"],
        df_silver["Outstanding_Amount"],
        df_silver["Duplicate_Flag"],
        df_product["Dim_Product_Key"],
        df_pharmacy["Dim_Pharmacy_Key"],
        df_channel["Dim_Channel_Key"],
        df_patient_zip["Dim_Patient_Key"],
        df_notes["Dim_Notes_Key"],
        df_plan["Dim_Plan_Key"],
        df_rep["Dim_Rep_Key"],
        df_submission["Dim_Submission_Key"],
    )
)


In [0]:
display(df_fact)

#### Writing Fact Table

In [0]:
from delta.tables import DeltaTable

In [0]:
from delta.tables import DeltaTable

if spark.catalog.tableExists("pharma_claim_craft.claim_craft_gold.fact_claimcraft"):
    deltatbl = DeltaTable.forName(
        spark, "pharma_claim_craft.claim_craft_gold.fact_claimcraft"
    )
    deltatbl.alias("trg").merge(
        df_fact.alias("src"),
        "trg.Dim_Product_Key=src.Dim_Product_Key AND trg.Dim_Pharmacy_Key=src.Dim_Pharmacy_Key AND trg.Dim_Channel_Key=src.Dim_Channel_Key AND trg.Dim_Patient_Key=src.Dim_Patient_Key AND trg.Dim_Notes_Key=src.Dim_Notes_Key AND trg.Dim_Plan_Key=src.Dim_Plan_Key AND trg.Dim_Rep_Key=src.Dim_Rep_Key AND trg.Dim_Submission_Key=src.Dim_Submission_Key",
    ).whenMatchedUpdateAll().whenNotMatchedInsertAll().execute()

else:
    df_fact.write.mode("overwrite").format("delta").option(
        "path", "abfss://gold@claimcraftadls.dfs.core.windows.net/fact_claimcraft"
    ).saveAsTable("pharma_claim_craft.claim_craft_gold.fact_claimcraft")

In [0]:
%sql
select * from `pharma_claim_craft`.claim_craft_gold.fact_claimcraft